## CodeLLaMA 34B Instruct GraphQL Generation Performance

In [2]:
API_KEY = "8c81391756ffaca12a60295b0130ea6377361dfe7b25169beea5ff22363f7d0b"

In [13]:
import requests

url = "https://api.together.xyz/inference"

payload = {
    "model": "codellama/CodeLlama-34b-Instruct-hf",
    "prompt": """Write a GraphQL Query for fetching names of all students from the given schema. Provide only the GraphQL query in the output.
    Schema: type student {
        id: ID!
        name: String
        class: String
        city: String
        phone: String
    }

    GraphQL Query: """,
    "max_tokens": 256,
    "stop": "\n\n",
    "temperature": 0.7,
    "top_p": 0.7,
    "top_k": 10,
    "repetition_penalty": 1
}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": "Bearer " + API_KEY
}    

response = requests.post(url, json=payload, headers=headers)
response.json()['output']['choices'][0]['text']

'\n    {\n        students {\n            name\n        }\n    }'

In [16]:
from datasets import load_dataset

dataset = load_dataset("xlangai/spider")

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [17]:
dataset['train']

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})

In [22]:
train_df = dataset['train'].to_pandas()
train_df

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."
...,...,...,...,...,...,...
6995,culture_company,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...,"[SELECT, T1.company_name, FROM, culture_compan...","[select, t1, ., company_name, from, culture_co...","[What, are, all, the, company, names, that, ha..."
6996,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[Show, the, movie, titles, and, book, titles, ..."
6997,culture_company,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...,"[SELECT, T1.title, ,, T3.book_title, FROM, mov...","[select, t1, ., title, ,, t3, ., book_title, f...","[What, are, the, titles, of, movies, and, book..."
6998,culture_company,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...,"[SELECT, T2.company_name, FROM, movie, AS, T1,...","[select, t2, ., company_name, from, movie, as,...","[Show, all, company, names, with, a, movie, di..."


In [51]:
def get_prompt(db, question, schema):
    return """Write a GraphQL Query from the given GraphQL schema based on the second question. I have provided one example query for the first question. Do not use any field that is not in the schema. Provide only the GraphQL query in the output.
Example Database: academic
Example Schema: ```type academic_student {
    id: ID!
    name: String
    class: String
    city: String
    phone: String
}```
Example Question: List the names of all students.
Example GraphQL Query: ```query {
    academic_student {
        name
    }
}```

Database: """ + db + """
Schema: ```""" + schema + """```
Question:  """ + question + """
GraphQL Query: """

In [52]:
eg = get_prompt("department_management", "List the ranking of all departments.", """type department_management_department {
    budget_in_billions: Float
    creation: String
    department_id: Int
    name: String
    num_employees: Float
    ranking: Int
}""")
print(eg)

Write a GraphQL Query from the given GraphQL schema based on the second question. I have provided one example query for the first question. Do not use any field that is not in the schema. Provide only the GraphQL query in the output.
Example Database: academic
Example Schema: ```type academic_student {
    id: ID!
    name: String
    class: String
    city: String
    phone: String
}```
Example Question: List the names of all students.
Example GraphQL Query: ```query {
    academic_student {
        name
    }
}```

Database: department_management
Schema: ```type department_management_department {
    budget_in_billions: Float
    creation: String
    department_id: Int
    name: String
    num_employees: Float
    ranking: Int
}```
Question:  List the ranking of all departments.
GraphQL Query: 


In [57]:
import requests

def run_inference(prompt):
    url = "https://api.together.xyz/inference"
    
    payload = {
        "model": "codellama/CodeLlama-34b-Instruct-hf",
        "prompt": prompt,
        "max_tokens": 256,
        "stop": "\n\n",
        "temperature": 0.1,
        "top_p": 0.7,
        "top_k": 10,
        "repetition_penalty": 1
    }
    
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": "Bearer " + API_KEY
    }    
    
    response = requests.post(url, json=payload, headers=headers)
    return response.json()['output']['choices'][0]['text']

In [58]:
x = run_inference(eg)
print(x)


```query {
    department_management_department {
        ranking
    }
}```
